# nn

> Fill in a module description here

In [ ]:
#| default_exp nn

In [ ]:
#| export
from typing import List, Callable, Any
from minima.autograd import Tensor
from minima import operators
import minima.init as init
import numpy as np
import torch

In [ ]:
#| export
class Parameter(Tensor):
    """
    A kind of Tensor that is to be considered a module parameter.

    Parameters are `Tensor` subclasses, that have a very special property when used with
    `Module` s - when they're assigned as Module attributes they are automatically added
    to the list of its parameters, and will appear in `Module.parameters()` iterator.
    Another difference is that parameters can't be volatile and that they require gradient by default.
    """

In [ ]:
#| export
def _unpack_params(value: object) -> List[Tensor]:
    """
    Unpack parameters from different Python objects.

    This function takes an object of type `Parameter`, `Module`, `dict`, `list`, or `tuple` and 
    recursively extracts any contained `Parameter` instances, returning them as a list. For other 
    object types, it returns an empty list.

    Args:
        value (object): The input object which could be of type `Parameter`, `Module`, `dict`, 
                        `list`, `tuple`, or any other type.

    Returns:
        List[Tensor]: A list containing all the `Parameter` instances found within the input object.
                      If no `Parameter` instances are found, an empty list is returned.
                      
    Example:
        module = nn.Module(...)
        params = _unpack_params(module)
        print(params)  # Prints list of `Parameter` instances contained in `module`.
    """
    if isinstance(value, Parameter):
        return [value]
    elif isinstance(value, Module):
        return list(value.parameters())
    elif isinstance(value, dict):
        return [item for v in value.values() for item in _unpack_params(v)]
    elif isinstance(value, (list, tuple)):
        return [item for v in value for item in _unpack_params(v)]
    return []

In [ ]:
#| export
def _child_modules(value: object) -> List["Module"]:
    """
    Recursively unpack child modules from different Python objects.

    This function takes an object of type `Module`, `dict`, `list`, or `tuple` and 
    recursively extracts any contained `Module` instances, returning them as a list. 
    For other object types, it returns an empty list.

    Args:
        value (object): The input object which could be of type `Module`, `dict`, 
                        `list`, `tuple`, or any other type.

    Returns:
        List[Module]: A list containing all the `Module` instances found within 
                      the input object. If no `Module` instances are found, 
                      an empty list is returned.

    Example:
        class MyModule(Module):
            def __init__(self):
                super().__init__()
                self.layer1 = nn.Linear(20, 20)
                self.layer2 = nn.Linear(20, 20)
        
        my_module = MyModule()
        children = _child_modules(my_module)
        print(children)  # Prints list of `Module` instances contained in `my_module`.
    """
    if isinstance(value, Module):
        return [value] + _child_modules(value.__dict__)
    elif isinstance(value, dict):
        return [item for v in value.values() for item in _child_modules(v)]
    elif isinstance(value, (list, tuple)):
        return [item for v in value for item in _child_modules(v)]
    else:
        return []

In [ ]:
#|export
class Module:
    
    def __init__(self):
        self.training = True
        
    def parameters(self) -> List[Parameter]:
        return _unpack_params(self.__dict__)
    
    def _children(self) -> List[Parameter]:
        return _child_modules(self.__dict__)
    
    def eval(self):
        self.training = False
        for m in self._children():
            m.training = False
            
    def train(self):
        self.training = True
        for m in self._children():
            m.training = True
            
    def __call__(self, *args, **kwargs):
        return self.forward(*args, **kwargs)

---
# minima.nn.Linear

The `minima.nn.Linear` class applies a linear transformation to the incoming data.

### Method

`minima.nn.Linear(in_features, out_features, bias=True, device=None, dtype="float32")`

#### Parameters

- **in_features** (int): The number of features (size of each input sample) from the input data.

- **out_features** (int): The number of features (size of each output sample) that the layer should produce.

- **bias** (bool, optional): If set to `False`, the layer will not learn an additive bias. Defaults to `True`.

- **device** (Device, optional): The device to run the computations on. If not specified, it will default to the device specified in the global context.

- **dtype** (str, optional): The data type of the output. Defaults to `"float32"`.

#### Returns

An instance of the `minima.nn.Linear` layer.

### Variables

- **weight** (Tensor): The learnable weights of the model, of shape (`in_features`, `out_features`). The values should be initialized using the Kaiming Uniform initialization method with `fan_in = in_features`.

- **bias** (Tensor): The learnable bias of the model, of shape (`out_features`). If the `bias` parameter is set to `True`, the values should be initialized using the Kaiming Uniform initialization method with `fan_in = out_features`.

### Details

The `minima.nn.Linear` class applies a linear transformation to the incoming data using the equation $y = xA^T + b$, where:
- $x$ is the input,
- $A^T$ is the transpose of the weight matrix,
- $b$ is the bias.

The input shape is expected to be $(N, H_{in})$ where $H_{in}$ is `in_features` and the output shape will be $(N, H_{out})$ where $H_{out}$ is `out_features`.

In [ ]:
#| export
class Linear(Module):
    """
    A class representing a fully connected (linear) layer in a neural network.
    This class inherits from the `Module` class.

    Attributes:
        in_features (int): The number of input features.
        out_features (int): The number of output features.
        device (str): The device to store the Parameters on (defaults to None, which means CPU).
        dtype (str): The data type of the Parameters (defaults to 'float32').
        weight (Parameter): The weight parameters of the layer.
        bias (Parameter): The bias parameters of the layer, or None if bias=False.

    Methods:
        forward(X: Tensor) -> Tensor: Compute the forward pass of the layer.
    """
    
    def __init__(self, in_features, out_features, bias=True, device=None, dtype="float32"):
        """
        Initialize the layer with given input/output feature sizes and, optionally, bias, device, and dtype.

        Args:
            in_features (int): The number of input features.
            out_features (int): The number of output features.
            bias (bool, optional): Whether or not to include a bias term. Default is True.
            device (str, optional): The device to store the Parameters on. Default is None, which means CPU.
            dtype (str, optional): The data type of the Parameters. Default is 'float32'.
        """
        
        super().__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.device = device
        self.dtype = dtype

        self.weight = Parameter(init.kaiming_uniform(fan_in=in_features, fan_out=out_features, device=device, dtype=dtype))
        self.bias = (Parameter(init.kaiming_uniform(fan_in=out_features, fan_out=1, device=device, dtype=dtype)).reshape((1, out_features))
                     if bias else None)
        
    def __repr__(self) -> str:
        return f'Linear(in_features={self.in_features}, out_features={self.out_features}, bias={self.bias is not None})'
            
    def forward(self, X: Tensor) -> Tensor:
        """
        Compute the forward pass of the layer.

        This function applies the linear transformation to the input tensor X, 
        i.e., performs the matrix multiplication of X and the weight tensor, 
        and then adds the bias tensor (if bias is not None).

        Args:
            X (Tensor): The input tensor.

        Returns:
            Tensor: The output tensor.
        """
        
        out = X @ self.weight
        out = out + self.bias.broadcast_to(out.shape) if self.bias else out

In [ ]:
class MyModule(Module):
    def __init__(self):
        super().__init__()
        self.layer1 = Linear(10, 20)
        self.layer2 = Linear(20, 10)

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        return x

my_module = MyModule()

children = _child_modules(my_module)
print(children)  # Prints list of `Module` instances contained in `my_module`.

[<__main__.MyModule object>, Linear(in_features=10, out_features=20, bias=True), Linear(in_features=20, out_features=10, bias=True)]


In [ ]:
Linear(in_features=10, out_features=5, bias=True)

Linear(in_features=10, out_features=5, bias=True)

## Export

In [ ]:
import nbdev; nbdev.nbdev_export()